# Exercício 5

Uma empresa construtora tem 5 tratores em locais diferentes e um trator é necessário para cada uma das três obras situadas em locais diferentes. Se os custos de transporte dos tratores forem os do quadro a seguir, determine o esquema de designação de custo mínimo.

| **LocTr\LocCons** 	| **A** 	| **B** 	| **C** 	|
|:-----------------:	|:-----:	|:-----:	|:-----:	|
|       **1**       	|   2   	|   3   	|   4   	|
|       **2**       	|   7   	|   6   	|   4   	|
|       **3**       	|   3   	|   5   	|   8   	|
|       **4**       	|   4   	|   6   	|   5   	|
|       **5**       	|   4   	|   6   	|   3   	|

# Solução

## Modelo Matemático

### Função Objetivo

$$ min \ Z = \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} $$

### Sujeito a

$$
\sum_{i \in I} x_{ij} = 1, \ \forall j \in J \ \dots \ (1)
$$

$$
\sum_{j \in J} x_{ij} \le 1, \ \forall i \in I \ \dots \ (2)
$$

$$
x_{ij} \in \set{0, 1}, \forall i \in I, \forall j \in J \ \dots \ (3)
$$

In [1]:
# --- Imports --- #
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [2]:
# --- Declaração dos Dados de Entrada --- #
# Matriz de custos
matriz_custos = np.array([[2, 3, 4],
                          [7, 6, 4],
                          [3, 5, 8],
                          [4, 6, 5],
                          [4, 6, 3]])
lista_tratores = ['T1', 'T2', 'T3', 'T4', 'T5'] # Lista de tratores
lista_construcoes = ['C1', 'C2', 'C3'] # Lista de construções
custo_designacao = {(lista_tratores[i], lista_construcoes[j]): matriz_custos[i, j] for i in range(len(lista_tratores)) for j in range(len(lista_construcoes))} # Dicionário de custos

In [3]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel() # Objeto do tipo ConcreteModel

In [4]:
# --- Declaração dos Conjuntos de Iteração --- #
modelo.tratores = pyo.Set(initialize=lista_tratores) # Conjunto de tratores
modelo.construcoes = pyo.Set(initialize=lista_construcoes) # Conjunto de construções

In [5]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.tratores, modelo.construcoes, within=pyo.Binary) # x_ij = 1 se o trator i for designado para a construção j, 0 caso contrário

In [6]:
# --- Declaração dos Parâmetros do Modelo --- #
modelo.custo_movimentacao = pyo.Param(modelo.tratores, modelo.construcoes, initialize=custo_designacao) # Custo de movimentação do trator i para a construção j

In [7]:
# --- Declaracao da Função Objetivo --- #
def funcao_objetivo(modelo):
    return sum(modelo.custo_movimentacao[i, j] * modelo.x[i, j] for i in modelo.tratores for j in modelo.construcoes) # Minimizar o custo total de movimentação
modelo.objetivo = pyo.Objective(rule=funcao_objetivo, sense=pyo.minimize) # Função objetivo

In [8]:
# --- Declaração das Restrições --- #
def restricao_tratores(modelo, i):
    return sum(modelo.x[i, :]) <= 1
modelo.rest_tratores = pyo.Constraint(modelo.tratores, rule=restricao_tratores) # Restrição de que cada trator deve ser designado para uma única construção
# ---
def restricao_construcoes(modelo, j):
    return sum(modelo.x[:, j]) == 1
modelo.rest_construcoes = pyo.Constraint(modelo.construcoes, rule=restricao_construcoes) # Restrição de que cada construção deve ser designada para um único trator

In [9]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi') # Solver Gurobi
solver.solve(modelo)

{'Problem': [{'Name': 'x1', 'Lower bound': 9.0, 'Upper bound': 9.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 15, 'Number of binary variables': 15, 'Number of integer variables': 15, 'Number of continuous variables': 0, 'Number of nonzeros': 30, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.016000032424926758', 'Error rc': 0, 'Time': 1.5269384384155273}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [16]:
print(f'Função Objetivo: {modelo.objetivo()}') # Função objetivo

Função Objetivo: 9.0


In [10]:
modelo.x.display()

x : Size=15, Index=tratores*construcoes
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('T1', 'C1') :     0 :   0.0 :     1 : False : False : Binary
    ('T1', 'C2') :     0 :   1.0 :     1 : False : False : Binary
    ('T1', 'C3') :     0 :  -0.0 :     1 : False : False : Binary
    ('T2', 'C1') :     0 :  -0.0 :     1 : False : False : Binary
    ('T2', 'C2') :     0 :  -0.0 :     1 : False : False : Binary
    ('T2', 'C3') :     0 :  -0.0 :     1 : False : False : Binary
    ('T3', 'C1') :     0 :   1.0 :     1 : False : False : Binary
    ('T3', 'C2') :     0 :  -0.0 :     1 : False : False : Binary
    ('T3', 'C3') :     0 :  -0.0 :     1 : False : False : Binary
    ('T4', 'C1') :     0 :  -0.0 :     1 : False : False : Binary
    ('T4', 'C2') :     0 :  -0.0 :     1 : False : False : Binary
    ('T4', 'C3') :     0 :  -0.0 :     1 : False : False : Binary
    ('T5', 'C1') :     0 :  -0.0 :     1 : False : False : Binary
    ('T5', 'C2') :     0 :  -0.0 :  

In [18]:
# --- Extração dos Resultados --- #
dados_designacao = [{'Trator': i, 'Construção': j, 'Designado': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_designacao).pivot(index='Trator', 
                                                  columns='Construção',
                                                  values='Designado')
# ---
resultados.to_excel('_ex_05_resultados_modelo_matematico.xlsx')